# **Fine-Tune FLAN-T5 with Reinforcement Learning (PPO) and PEFT to Generate Less-Toxic Summaries**

Fine-Tuning a `FLAN-T5` model using **Proximal Policy Optimization (PPO)** and **Parameter-Efficient Fine-Tuning (PEFT)** to generate **less toxic summaries** of dialogues. A **reward model** based on Meta AI’s **RoBERTa hate speech classifier** is used to guide the fine-tuning process. The key objective is to encourage the model to produce **non-toxic, safe summaries**.

### Goal

- Load a summarization dataset (`DialogSum`) and a pre-trained `FLAN-T5` model.
- Use a **hate speech classifier** as a **reward model** to guide detoxification.
- Apply **Reinforcement Learning with PPO** on top of **LoRA-based PEFT**.
- Compare toxicity scores before and after fine-tuning using Hugging Face `evaluate`.


Import the necessary packages.

In [3]:
# !pip install transformers datasets peft trl torch evaluate numpy pandas tqdm

In [4]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM, GenerationConfig
from datasets import load_dataset, load_from_disk
from peft import PeftModel, PeftConfig, LoraConfig, TaskType

# trl: Transformer Reinforcement Learning library
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import torch
import evaluate

import numpy as np
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

## Load FLAN-T5 Model, Fine-Tuned Peft Adapter,  Prepare Reward Model and Toxicity Evaluator

### Load Data and FLAN-T5 Model Fine-Tuned with Summarization Instruction

---



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
model_path = "flan-t5-base"
dataset_path = "dialogsum_dataset"

dataset_original = load_from_disk(dataset_path)
dataset_original

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [10]:
sample = dataset_original["train"][0]
print(sample["dialogue"])
print("-" * 100)
print(sample["summary"])
print("-" * 100)
print(sample["topic"])

#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?
#Person2#: I found it would be a good idea to get a check-up.
#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.
#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?
#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.
#Person2#: Ok.
#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?
#Person2#: Yes.
#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.
#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.
#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.
#Person2#: Ok, thanks doctor.
--------------------------------------------------------------------

Preprocessing the dataset. Wraping each dialogue with the instruction and tokenize the prompts. Save the token ids in the field `input_ids` and decoded version of the prompts in the field `query`.

In [18]:
# tokenizer = AutoTokenizer.from_pretrained('flan-t5-base/', device_map=device)

In [19]:
# tokenizer.encode("How are you?")

In [20]:
# print(tokenizer.decode([571, 33, 25, 58, 1], skip_special_tokens=True))

In [21]:
def build_dataset(model_path,
                  dataset_path):

    """
    Preprocess the dataset

    Parameters:
    - model_name (str): Tokenizer model name.
    - dataset_name (str): Name of the dataset to load.

    Returns:
    - dataset_splits (datasets.dataset_dict.DatasetDict): Preprocessed dataset containing train val test parts.
    """

    dataset = load_from_disk(dataset_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path, device_map=device)

    def tokenize(sample):

        prompt = f"""
Summarize the following conversation.

{sample["dialogue"]}

Summary:
"""
        sample["input_ids"] = tokenizer.encode(prompt, padding="max_length", truncation=True, max_length=512)


        # This must be called "query", which is a requirement of PPO library.
        sample["query"] = tokenizer.decode(sample["input_ids"])
        return sample

    for split in dataset.keys():
        dataset[split] = dataset[split].map(tokenize)
        dataset[split].set_format(type="torch", columns=["input_ids", "query"])

    return dataset

In [22]:
dataset = build_dataset(model_path=model_path, dataset_path=dataset_path)
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic', 'input_ids', 'query'],
        num_rows: 1500
    })
})


In [27]:
sample = dataset["train"][0]
print(sample["input_ids"])
print("-" * 100)
print(sample["query"])

tensor([12198,  1635,  1737,     8,   826,  3634,     5,  1713,   345, 13515,
          536,  4663,    10,  2018,     6,  1363,     5,  3931,     5,    27,
           31,    51,  7582, 12833,    77,     7,     5,  1615,    33,    25,
          270,   469,    58,  1713,   345, 13515,   357,  4663,    10,    27,
          435,    34,   133,    36,     3,     9,   207,   800,    12,   129,
            3,     9,   691,    18,   413,     5,  1713,   345, 13515,   536,
         4663,    10,  2163,     6,   168,     6,    25,    43,    29,    31,
           17,   141,    80,    21,   305,   203,     5,   148,   225,    43,
           80,   334,   215,     5,  1713,   345, 13515,   357,  4663,    10,
           27,   214,     5,    27,  2320,    38,   307,    38,   132,    19,
         1327,  1786,     6,   572,   281,   217,     8,  2472,    58,  1713,
          345, 13515,   536,  4663,    10,  1548,     6,     8,   200,   194,
           12,  1792,  2261, 21154,    19,    12,   253,    91, 

Let's load the peft model trained in a GPU environment

Adding the fine-tuned adapter to the original FLAN-T5 model. In the fine tuning with peft phase, the fully trained adapter was added only for inferences, so there was no need to pass LoRA configurations doing that. Now there is need to pass them to the constructed PEFT model, also putting `is_trainable=True`.

**The base model (e.g. flan-t5-base) + LoRA adapter = fine-tuned policy model.**

In [28]:
lora_config = LoraConfig(
    r=32,                            # Rank of the low-rank adaptation matrices
    lora_alpha=32,                   # Scaling factor for LoRA weights
    target_modules=["q", "v"],       # Apply LoRA only to these modules (query, value in attention)
    lora_dropout=0.05,               # Dropout applied to LoRA layers during training
    bias="none",                     # Whether to train bias terms
    task_type=TaskType.SEQ_2_SEQ_LM  # Task type: Sequence-to-Sequence Language Modeling
)

model = AutoModelForSeq2SeqLM.from_pretrained(model_path,
                                              torch_dtype=torch.bfloat16).to(device)  # torch.bfloat16 to reduce memory

peft_model = PeftModel.from_pretrained(model,
                                       'peft_ft',    # fine-tuned LoRA adapter
                                       lora_config=lora_config,
                                       torch_dtype=torch.bfloat16,
                                       device_map=device,
                                       is_trainable=True)     # can update LoRA

pull out the number of model parameters

In [29]:
# total and trainable parameters
total_params = sum(p.numel() for p in peft_model.parameters())
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")

Total parameters: 251116800
Trainable parameters: 3538944
Trainable %: 1.41%


Preparing to fine-tune the LLM using Reinforcement Learning (RL). Preparing the Proximal Policy Optimization (PPO) model passing the instruct-fine-tuned PEFT model to it. PPO will be used to optimize the RL policy against the reward model.

**A new value head used by PPO to estimate expected reward**

In [30]:
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(peft_model,   # fine-tuned LoRA policy model
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True).to(device) # PPO can update LoRA + value head

print(f'PPO model parameters to be updated (ValueHead + 769 params):\n')
# total and trainable parameters
total_params = sum(p.numel() for p in ppo_model.parameters())
trainable_params = sum(p.numel() for p in ppo_model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")
print(ppo_model.v_head)

PPO model parameters to be updated (ValueHead + 769 params):

Total parameters: 251117569
Trainable parameters: 3539713
Trainable %: 1.41%
ValueHead(
  (dropout): Dropout(p=0.1, inplace=False)
  (summary): Linear(in_features=768, out_features=1, bias=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


During PPO, only a few parameters will be updated. Specifically, the parameters of the `ValueHead`. More information about this class of models can be found in the [documentation](https://huggingface.co/docs/trl/main/en/models#trl.create_reference_model). The number of trainable parameters computed as $(n+1)*m$, where $n$ is the number of input units (here $n=768$) and $m$ is the number of output units (you have $m=1$). The $+1$ term in the equation takes into account the bias term.

In [31]:
3538944 + 769 # Trainable

3539713

Creating a frozen copy of the PPO which will not be fine-tuned - a reference model. The reference model will represent the LLM before detoxification. None of the parameters of the reference model will be updated during PPO training.

In PPO (Proximal Policy Optimization) with RLHF, we need two models:

- ppo_model – the policy model WE're training (LoRA + value head)
- ref_model – the reference model used to compute the KL divergence (a penalty if the new model strays too far from the original)

In [32]:
ref_model = create_reference_model(ppo_model)

total_params = sum(p.numel() for p in ref_model.parameters())
trainable_params = sum(p.numel() for p in ref_model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")
print(f"Trainable %: {100 * trainable_params / total_params:.2f}%")

Total parameters: 251117569
Trainable parameters: 0
Trainable %: 0.00%


Everything is set. It is time to prepare the reward model!

### Prepare Reward Model

**Reinforcement Learning (RL)** is one type of machine learning where agents take actions in an environment aimed at maximizing their cumulative rewards. The agent's behavior is defined by the **policy**. And the goal of reinforcement learning is for the agent to learn an optimal, or nearly-optimal, policy that maximizes the **reward function**.

The original policy is based on the instruct PEFT model - this is the LLM before detoxification and it was trained on dialogue summaries dataset in a GPU environment. We ask human labelers to give feedback on the outputs' toxicity. However, it can be expensive to use them for the entire fine-tuning process. A practical way to avoid that is to use a reward model encouraging the agent to detoxify the dialogue summaries. The intuitive approach would be to do some form of sentiment analysis across two classes (`nothate` and `hate`) and give a higher reward if there is higher a chance of getting class `nothate` as an output.

Having human labelers for the entire finetuning process can be expensive. A practical way to avoid that is to use a reward model.
Use feedback generated by a model

We will use [Meta AI's RoBERTa-based hate speech model](https://huggingface.co/facebook/roberta-hate-speech-dynabench-r4-target) for the reward model. This model will output **logits** and then predict probabilities across two classes: `nothate` and `hate`. The logits of the output `nothate` will be taken as a positive reward. Then, the model will be fine-tuned with PPO using those reward values.

Creating the instance of the required model class for the RoBERTa model. Loading a tokenizer to test the model. The model label `0` will correspond to the class `nothate` and label `1` to the class `hate`.

**Purpose: For every generated summary, this model gives a score. The score for "nothate" is our reward signal.**

In [142]:
toxicity_model_name = "toxicity-roberta/"
toxicity_tokenizer = AutoTokenizer.from_pretrained(toxicity_model_name, device_map=device)
toxicity_model = AutoModelForSequenceClassification.from_pretrained(toxicity_model_name, device_map=device)
print(toxicity_model.config.id2label)

{0: 'nothate', 1: 'hate'}


Taking some non-toxic text, tokenize it, and pass it to the model. Printing the output logits, probabilities, and the corresponding reward that will be used for fine-tuning.

In [143]:
not_hate_index = 0

In [144]:
non_toxic_text = "#Person 1# tells Tommy that he didn't like the movie."
toxicity_input_ids = toxicity_tokenizer(non_toxic_text, return_tensors="pt").input_ids
toxicity_input_ids = toxicity_input_ids.to(device)

logits = toxicity_model(input_ids=toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# get the logits for "not hate" - this is the reward!
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (high): {nothate_reward}')

logits [not hate, hate]: [3.114103078842163, -2.4896199703216553]
probabilities [not hate, hate]: [0.9963293671607971, 0.003670599078759551]
reward (high): [3.114103078842163]


Let's show a toxic comment.  This will have a low reward because it is more toxic.

In [145]:
toxic_text = "#Person 1# tells Tommy that the movie was terrible, dumb and stupid."
toxicity_input_ids = toxicity_tokenizer(toxic_text, return_tensors="pt").input_ids
toxicity_input_ids = toxicity_input_ids.to(device)

logits = toxicity_model(toxicity_input_ids).logits
print(f'logits [not hate, hate]: {logits.tolist()[0]}')

# Print the probabilities for [not hate, hate]
probabilities = logits.softmax(dim=-1).tolist()[0]
print(f'probabilities [not hate, hate]: {probabilities}')

# Get the logits for "not hate" - this is the reward!
nothate_reward = (logits[:, not_hate_index]).tolist()
print(f'reward (low): {nothate_reward}')

logits [not hate, hate]: [-0.6921160817146301, 0.3722703754901886]
probabilities [not hate, hate]: [0.25647208094596863, 0.743527889251709]
reward (low): [-0.6921160817146301]


How This Fits into Phase 3 (PPO)
In the final RLHF phase, the PPO trainer will:

- Use SFT model (peft ft) to generate a summary.

- Feed that summary to this toxicity_model to get a reward.

- Update SFT model to teach it to generate summaries that get a high score from the toxicity judge.

The end result will be a summarization model that is actively optimized to avoid producing toxic content.

Setup Hugging Face inference pipeline to simplify the code for the toxicity reward model:

In [146]:
sentiment_pipe = pipeline("sentiment-analysis",
                          model=toxicity_model_name,
                          device=device,
                          framework="pt")

reward_logits_kwargs = {
    "top_k": None,                  # Return all scores.
    "function_to_apply": "none",    # Set to "none" to retrieve raw logits.
    "batch_size": 16
}

reward_probabilities_kwargs = {
    "top_k": None,                  # Return all scores.
    "function_to_apply": "softmax", # Set to "softmax" to apply softmax and retrieve probabilities.
    "batch_size": 16
}

print("Reward model output:")
print("For non-toxic text")
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))
print("For toxic text")
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

Reward model output:
For non-toxic text
[{'label': 'nothate', 'score': 3.114103078842163}, {'label': 'hate', 'score': -2.4896199703216553}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.0036705993115901947}]
For toxic text
[{'label': 'hate', 'score': 0.3722703754901886}, {'label': 'nothate', 'score': -0.6921160817146301}]
[{'label': 'hate', 'score': 0.743527889251709}, {'label': 'nothate', 'score': 0.25647208094596863}]


The outputs are the logits for both `nothate` (positive) and `hate` (negative) classes. But PPO will be using logits only of the `nothate` class as the positive reward signal used to help detoxify the LLM outputs.

In [147]:
print(sentiment_pipe(non_toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(non_toxic_text, **reward_probabilities_kwargs))

[{'label': 'nothate', 'score': 3.114103078842163}, {'label': 'hate', 'score': -2.4896199703216553}]
[{'label': 'nothate', 'score': 0.9963293671607971}, {'label': 'hate', 'score': 0.0036705993115901947}]


In [148]:
print(sentiment_pipe(toxic_text, **reward_logits_kwargs))
print(sentiment_pipe(toxic_text, **reward_probabilities_kwargs))

[{'label': 'hate', 'score': 0.3722703754901886}, {'label': 'nothate', 'score': -0.6921160817146301}]
[{'label': 'hate', 'score': 0.743527889251709}, {'label': 'nothate', 'score': 0.25647208094596863}]


### Evaluate Toxicity

To evaluate the model before and after fine-tuning/detoxification, Setting up the [toxicity evaluation metric](https://huggingface.co/spaces/evaluate-measurement/toxicity). The **toxicity score** is a decimal value between 0 and 1 where 1 is the highest toxicity.

In [149]:
def get_toxicity_score(texts):
    if isinstance(texts, str):
        texts = [texts]
    inputs = toxicity_tokenizer(texts, return_tensors="pt", truncation=True, padding=True).to(toxicity_model.device)
    with torch.no_grad():
        outputs = toxicity_model(**inputs)
        scores = torch.softmax(outputs.logits, dim=-1)
    return scores[:, 1].tolist()  # Return list of "hate" class scores


print(get_toxicity_score(non_toxic_text))
print(get_toxicity_score(toxic_text))

[0.003670599078759551]
[0.743527889251709]


In [43]:
# toxicity_evaluator = evaluate.load("toxicity",
#                                     'facebook/roberta-hate-speech-dynabench-r4-target',
#                                     module_type="measurement",
#                                     toxic_label="hate")

Calculating toxicity for the same sentences. Toxicity scores are the probabilities of `hate` class returned directly from the reward model.

In [70]:
# toxicity_score = toxicity_evaluator.compute(predictions=[
#     non_toxic_text
# ])
# print("Toxicity score for non-toxic text:")
# print(toxicity_score["toxicity"])

# toxicity_score = toxicity_evaluator.compute(predictions=[
#     toxic_text
# ])
# print("\nToxicity score for toxic text:")
# print(toxicity_score["toxicity"])

This evaluator can be used to compute the toxicity of the dialogues. Need to pass the test dataset (`dataset["test"]`), the same tokenizer which was used before, the frozen PEFT model prepared before, and the toxicity evaluator. It is convenient to wrap the required steps in the function `evaluate_toxicity`.

In [150]:
def evaluate_toxicity(model,
                      toxicity_evaluator,
                      tokenizer,
                      dataset,
                      num_samples):

    """
    Evaluate the toxicity of the model.

    Parameters:
    - model (trl model): Model to be evaluated.
    - toxicity_evaluator (evaluate_modules toxicity metrics): Toxicity evaluator.
    - tokenizer (transformers tokenizer): Tokenizer to be used.
    - dataset (dataset): Input dataset for the evaluation.
    - num_samples (int): Maximum number of samples for the evaluation.

    Returns:
    tuple: A tuple containing two numpy.float64 values:
    - mean (numpy.float64): Mean of the samples toxicity.
    - std (numpy.float64): Standard deviation of the samples toxicity.
    """

    max_new_tokens=100

    toxicities = []
    input_texts = []
    for i, sample in tqdm(enumerate(dataset)):
        input_text = sample["query"]

        if i > num_samples:
            break

        input_ids = tokenizer(input_text, padding="max_length", truncation=True, max_length=512, return_tensors="pt").input_ids
        input_ids = input_ids.to(device)

        generation_config = GenerationConfig(max_new_tokens=max_new_tokens,
                                             top_k=0.0,
                                             top_p=1.0,
                                             do_sample=True)

        response_token_ids = model.generate(input_ids=input_ids,
                                            generation_config=generation_config)

        generated_text = tokenizer.decode(response_token_ids[0], skip_special_tokens=True)

        toxicity_score = toxicity_evaluator([input_text + " " + generated_text])

        toxicities.append(toxicity_score)

    # Compute mean & std using np.
#     print(toxicities)
    mean = np.mean(toxicities)
    std = np.std(toxicities)

    return mean, std

Performing the calculation of the model toxicity before fine-tuning/detoxification:

In [65]:
tokenizer = AutoTokenizer.from_pretrained(model_path, device_map=device)

mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model,
                                                                          toxicity_evaluator=get_toxicity_score,
                                                                          tokenizer=tokenizer,
                                                                          dataset=dataset["test"],
                                                                          num_samples=50)

print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')

51it [00:59,  1.16s/it]

toxicity [mean, std] before detox: [0.155942528782522, 0.061500386888910345]


## Performing Fine-Tuning to Detoxify the Summaries
Optimizing a RL policy against the reward model using Proximal Policy Optimization (PPO).

### Initialize `PPOTrainer`

For the `PPOTrainer` initialization, we will need a collator. Here it will be a function transforming the dictionaries in a particular way. We can define and test it:

In [66]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}]
Collator output: {'key1': ['value1'], 'key2': ['value2'], 'key3': ['value3']}


Setting up the configuration parameters. Loading the `ppo_model` and the tokenizer. We will also load a frozen version of the model `ref_model`. The first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This works as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original LLM.

In [67]:
learning_rate=1.41e-5
max_ppo_epochs=1
mini_batch_size=4
batch_size=16

config = PPOConfig(
    model_name=model_path,    # for logging
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

ppo_trainer = PPOTrainer(config=config,
                         model=ppo_model,     # model that will generate summaries and be updated.
                         ref_model=ref_model,
                         tokenizer=tokenizer,
                         dataset=dataset["train"],
                         data_collator=collator)

In [70]:
# Just check first few batches to avoid printing everything
for i, batch in enumerate(tqdm(ppo_trainer.dataloader)):
    print(f"\nBatch {i+1} -----------------------------")
    for key, value in batch.items():
        print(f"{key}:")
        print(f"  Type: {type(value)}")
        print(f"  Length: {len(value)}")
        print(f"  Sample: {value[0] if len(value) > 0 else 'Empty'}")
    
    if i >= 0:  # Print only first 1 batches
        break

  0%|          | 0/778 [00:00<?, ?it/s]


Batch 1 -----------------------------
input_ids:
  Type: <class 'list'>
  Length: 16
  Sample: tensor([12198,  1635,  1737,     8,   826,  3634,     5,  1713,   345, 13515,
          536,  4663,    10,  1804,  1379,     6,   283,  5805,  1589,     5,
         1713,   345, 13515,   357,  4663,    10,  1804,  1379,     5,  9348,
           25,   199,   140,   754,    58,    27,    31,    51,   479,    21,
          128,  1335,    21,    82,  2039,     5,  1713,   345, 13515,   536,
         4663,    10,  6902,     5,   363,   773,    13,  1335,    19,   255,
         1638,    16,    58,  1713,   345, 13515,   357,  4663,    10,   451,
           31,     7,   182,  3036,    13,  7966,   333,  1937,     5,  1713,
          345, 13515,   536,  4663,    10,    27,   217,     5,   363,    81,
           48,    80,    58,  4498,   255,   608,    34,   274,    58,  1713,
          345, 13515,   357,  4663,    10,    27,    31,    51,    59,   417,
            5,   299,   255,  1077,   751,    

### Fine-Tuning the Model

The fine-tuning loop consists of the following main steps:
1. Get the query responses from the policy LLM (PEFT model).
2. Get sentiments for query/responses from hate speech RoBERTa model.
3. Optimize policy with PPO using the (query, response, reward) triplet.

The operation is running if we see the following metrics appearing:
* `objective/kl`: minimize kl divergence,
* `ppo/returns/mean`: maximize mean returns,
* `ppo/policy/advantages_mean`: maximize advantages.

In [76]:
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

generation_kwargs = {
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

reward_kwargs = {
    "top_k": None,               # Return all scores
    "function_to_apply": "none", # Get raw logits
    "batch_size": 16             # Batch inference for GPU efficiency
}

max_ppo_steps = 50

for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if step >= max_ppo_steps:
        break

    prompt_tensors = batch["input_ids"]

    # Generate summaries (responses) using the policy model
    summary_tensors = []
    for prompt_tensor in prompt_tensors:
        max_new_tokens = output_length_sampler()
        generation_kwargs["max_new_tokens"] = max_new_tokens

        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    # Decode the generated summaries
    batch["response"] = [tokenizer.decode(r.squeeze(), skip_special_tokens=True) for r in summary_tensors]

    # Build joint input (query + response) for reward model
    MAX_SEQ_LEN = 512
    query_response_pairs = [q + r for q, r in zip(batch["query"], batch["response"])]

    # Truncate if necessary and tokenize
    query_response_pairs = [
        tokenizer.decode(tokenizer(qr, truncation=True, max_length=MAX_SEQ_LEN)["input_ids"])
        for qr in query_response_pairs
    ]

    # Use reward model (e.g. Meta AI's hate speech model) to get toxicity scores
    rewards = sentiment_pipe(query_response_pairs, **reward_kwargs)
    reward_tensors = [torch.tensor(reward[not_hate_index]["score"]) for reward in rewards]

    # Run PPO update
    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    # Log key metrics
    print(f'objective/kl: {stats["objective/kl"]}')
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}')
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}')
    print('-' * 100)


1it [00:22, 22.64s/it]

objective/kl: 29.8692569732666
ppo/returns/mean: -0.7105279564857483
ppo/policy/advantages_mean: 0.03865646570920944
----------------------------------------------------------------------------------------------------


2it [00:45, 22.61s/it]

objective/kl: 27.942033767700195
ppo/returns/mean: -0.5913649797439575
ppo/policy/advantages_mean: -0.01949869468808174
----------------------------------------------------------------------------------------------------


3it [01:08, 22.84s/it]

...

objective/kl: -341.1752014160156
ppo/returns/mean: 23.607318878173828
ppo/policy/advantages_mean: 0.01569041609764099
----------------------------------------------------------------------------------------------------


49it [17:14, 20.77s/it]

objective/kl: 23.669525146484375
ppo/returns/mean: -0.4850413501262665
ppo/policy/advantages_mean: 0.015226350165903568
----------------------------------------------------------------------------------------------------


50it [17:35, 21.12s/it]

objective/kl: 19.973003387451172
ppo/returns/mean: -0.2658303678035736
ppo/policy/advantages_mean: 0.025714825838804245
----------------------------------------------------------------------------------------------------


In [78]:
# Create save directory
import os
save_path = "./ppo_finetuned_model"
os.makedirs(save_path, exist_ok=True)

# Save the model (policy model after PPO fine-tuning)
ppo_trainer.model.save_pretrained(save_path)

# Save tokenizer
tokenizer.save_pretrained(save_path)

('./ppo_finetuned_model/tokenizer_config.json',
 './ppo_finetuned_model/special_tokens_map.json',
 './ppo_finetuned_model/tokenizer.json')

### Evaluate the Model Quantitatively

Load the PPO/PEFT model back in from disk and use the test dataset split to evaluate the toxicity score of the RL-fine-tuned model.

In [160]:
tokenizer = AutoTokenizer.from_pretrained(model_path, device_map=device)

mean_before_detoxification, std_before_detoxification = evaluate_toxicity(model=ref_model,
                                                                          toxicity_evaluator=get_toxicity_score,
                                                                          tokenizer=tokenizer,
                                                                          dataset=dataset["test"],
                                                                          num_samples=10)

print(f'toxicity [mean, std] before detox: [{mean_before_detoxification}, {std_before_detoxification}]')

11it [00:12,  1.13s/it]

toxicity [mean, std] before detox: [0.11854438839310949, 0.07926753609766712]


In [163]:
mean_after_detoxification, std_after_detoxification = evaluate_toxicity(model=ppo_model,
                                                                        toxicity_evaluator=get_toxicity_score,
                                                                        tokenizer=tokenizer,
                                                                        dataset=dataset["test"],
                                                                        num_samples=10)
print(f'toxicity [mean, std] after detox: [{mean_after_detoxification}, {std_after_detoxification}]')

11it [00:12,  1.13s/it]

toxicity [mean, std] after detox: [0.10763244356282732, 0.08202753552020547]


And compare the toxicity scores of the reference model (before detoxification) and fine-tuned model (after detoxification).

In [164]:
mean_improvement = (mean_before_detoxification - mean_after_detoxification) / mean_before_detoxification
std_improvement = (std_before_detoxification - std_after_detoxification) / std_before_detoxification

print(f'Percentage improvement of toxicity score after detoxification:')
print(f'mean: {mean_improvement*100:.2f}%')
print(f'std: {std_improvement*100:.2f}%')

Percentage improvement of toxicity score after detoxification:
mean: 9.20%
std: -3.48%


### Evaluate the Model Qualitatively

We can compare the original `ref_model` to the fine-tuned/detoxified `ppo_model` using the toxicity evaluator.

In [165]:
batch_size = 20
compare_results = {}

df_batch = dataset["test"][0:batch_size]

compare_results["query"] = df_batch["query"]
prompt_tensors = df_batch["input_ids"]

summary_tensors_ref = []
summary_tensors = []

# Get response from ppo and base model.
for i in tqdm(range(batch_size)):
    gen_len = output_length_sampler()
    generation_kwargs["max_new_tokens"] = gen_len

    summary = ref_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors_ref.append(summary)

    summary = ppo_model.generate(
        input_ids=torch.as_tensor(prompt_tensors[i]).unsqueeze(dim=0).to(device),
        **generation_kwargs
    ).squeeze()[-gen_len:]
    summary_tensors.append(summary)

# Decode responses.
compare_results["response_before"] = [tokenizer.decode(summary_tensors_ref[i]) for i in range(batch_size)]
compare_results["response_after"] = [tokenizer.decode(summary_tensors[i]) for i in range(batch_size)]

# Sentiment analysis of query/response pairs before/after.
MAX_LEN = 512  # Or 514 depending on your model

texts_before = [d + s for d, s in zip(compare_results["query"], compare_results["response_before"])]
texts_before = [
    tokenizer.decode(tokenizer(t, truncation=True, max_length=MAX_LEN)["input_ids"])
    for t in texts_before
]
rewards_before = sentiment_pipe(texts_before, **reward_kwargs)
compare_results["reward_before"] = [reward[not_hate_index]["score"] for reward in rewards_before]

texts_after = [d + s for d, s in zip(compare_results["query"], compare_results["response_after"])]
texts_after = [
    tokenizer.decode(tokenizer(t, truncation=True, max_length=MAX_LEN)["input_ids"])
    for t in texts_after
]
rewards_after = sentiment_pipe(texts_after, **reward_kwargs)

compare_results["reward_after"] = [reward[not_hate_index]["score"] for reward in rewards_after]

100%|██████████| 20/20 [00:51<00:00,  2.56s/it]


Store and review the results in a DataFrame

In [84]:
pd.set_option('display.max_colwidth', 500)
df_compare_results = pd.DataFrame(compare_results)
df_compare_results["reward_diff"] = df_compare_results['reward_after'] - df_compare_results['reward_before']
df_compare_results_sorted = df_compare_results.sort_values(by=['reward_diff'], ascending=False).reset_index(drop=True)
df_compare_results_sorted

,query,response_before,response_after,reward_before,reward_after,reward_diff
0,"Summarize the following conversation. #Person1#: Ms. Dawson, I need you to take a dictation for me. #Person2#: Yes, sir... #Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready? #Person2#: Yes, sir. Go ahead. #Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibi...",<pad> Ms. Dawson takes a dictation of new and restrictive procedures to all employees.</s>,<pad> #Person1# wants to tell Ms. Dawson to take a dictation of all employee mail. The code for admitting the communication between employees is first defined by the mediums used and the article to be used during working on the document between employees and the employees. The full code makes sure that the employee who can use Instant Messaging before #4 counters court. Mr. Dawson encourages this.</s>,1.854992,1.854992,0.0
1,"Summarize the following conversation. #Person1#: Ms. Dawson, I need you to take a dictation for me. #Person2#: Yes, sir... #Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready? #Person2#: Yes, sir. Go ahead. #Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibi...","<pad> #Person1# asks Ms. Dawson to take a dictation to all employees by this afternoon. To make things clear it should apply to intra-office communications only and to general communications, including outside communications and international communications, the memo will not be enforced. Cull the memo be typed up and distributed.</s>","<pad> Ms. Dawson gives details to staff, encourages use of instant messaging, and encourages staff to regular the wording. The department head reveals one new practice to most sports athletes.</s>",1.854992,1.854992,0.0
2,Summarize the following conversation. #Person1#: What's wrong with you? Why are you scratching so much? #Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak. #Person1#: Let me have a look. Whoa! Get away from me! #Person2#: What's wrong? #Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me! #Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor. #Person...,"<pad> #Person2# has chicken pox. #Person1# thinks #Person2# have chicken pox, but #Person2# shows signs. #Person1# takes a bath on Friday morning.</s>",<pad> #Person1# thinks #Person2# has chicken pox but #Person2# tells #Person1# this is a biohazard. You really burn!</s>,0.771682,0.771682,0.0
3,"Summarize the following conversation. #Person1#: I've had it! I am done working for a company that is taking me nowhere! #Person2#: So what are you gonna do? Just quit? #Person1#: That's exactly what I am going to do! I have decided to create my own company! I am going to write up a business plan, get some investors and start working for myself! #Person2#: Have you ever written up a business plan before? #Person1#: Well, no, it can't be that hard! I mean, all you have to do is explain your b...","<pad> #Person1# recently quit following a company that is taking #Person1# nowhere. #Person1# writes a business plan and will start work for himself. #Person2# tells #Person1# to write down a business plan and give away important information like the market analysis, the land policy, forecast earnings and profits. #Person2# feels the most important piece of information needed for the investors are the financial analysis.</s>",<pad> #Person1# decides to start being a business company and wants to make it more accessible. #Person1

**Looking at the reward mean/median of the generated sequences we can observe a significant difference!**